# Smart Drilhole: Prototype DDL with Core

In this section we create a drillhole database using SQLAlchemy. The idea is to reuse the function to create the same database in SQLITE and MySQL. 

Restrictions: 
- use ``PRAGMA foreign_keys = ON;`` in SQLITE
- use ``PRAGMA case_sensitive_like = True;`` in SQLITE
- use InnoDB in MySQL

In [2]:
import sqlalchemy
sqlalchemy.__version__ 

'1.1.9'

In [3]:
from sqlalchemy import (create_engine, 
                       Table, 
                       Column, 
                       Float,  
                       String, 
                       MetaData, 
                       ForeignKey, 
                       CheckConstraint)


def og_connect(con_string= 'sqlite:///test2.sqlite', overwrite = False, echo=True): 
    
    print 'connection string:', con_string
    print 'echo', echo
    
    
    # get database type
    dbtype=con_string[0:5]
        
    if dbtype=='sqlit':
        
        # TODO: check database is empty
        
        # Do a row connection and update some pragma
        eng = create_engine(con_string, echo=echo)
        with eng.connect() as con:
            print con.execute('PRAGMA foreign_keys = ON;') 
            print con.execute('PRAGMA case_sensitive_like = True;')
            
    if dbtype=='postg':
        eng = create_engine(con_string, echo=echo)
    
    if dbtype=='sqlite':
        pass
    
    #create collar table
    meta = MetaData()
    collar = Table('collar', meta,
                    Column('BHID', String, primary_key=True),
                    Column('xcollar', Float, nullable=False),
                    Column('ycollar', Float, nullable=False),
                    Column('zcollar', Float, nullable=False),
                    Column('LENGTH', Float, nullable=False),
                    Column('Comments', String))
    
    survey = Table('survey', meta,
                    Column('BHID', None, 
                           ForeignKey(column='collar.BHID', 
                             ondelete='CASCADE', 
                             onupdate='CASCADE',
                             name = 'chk_bhid'),
                           primary_key=True),
                    Column('at', Float, nullable=False, primary_key=True),
                    Column('az', Float, nullable=False),
                    Column('dip', Float, nullable=False),
                    Column('Comments', String))
    
    
    return eng, meta

def og_add_interval(eng, meta, table_name, cols={}):
       
    # create interval table       
    interval = Table(table_name, meta,
                    Column('BHID', None, 
                           ForeignKey(column='collar.BHID', 
                             ondelete='CASCADE', 
                             onupdate='CASCADE',
                             name = 'chk_bhid'),
                           primary_key=True),
                    Column('FROM', Float, nullable=False, primary_key=True),
                    Column('TO', Float, nullable=False),
                    Column('Comments', String),
                    CheckConstraint('"TO" > "FROM"', name='check_interv'))
    
    for col in cols:
        if 'foreignkey' in cols[col]:
            fk = ForeignKey (column= cols[col]['foreignkey']['column'],
                             ondelete= cols[col]['foreignkey']['ondelete'],
                             onupdate= cols[col]['foreignkey']['onupdate'])
            tmpcol = Column(col, None, fk)
        else:
            tmpcol = Column(col, cols[col]['coltypes'], nullable=cols[col]['nullable'])
        
        interval.append_column(tmpcol)

def og_references(eng, meta, table_name = 'assay_certificate', key = 'SampleID', cols={}):
       
    # create interval table       
    interval = Table(table_name, meta,
                    Column(key, String, primary_key=True),
                    Column('Comments', String)) 
    for col in cols:
        tmpcol = Column(col, cols[col]['coltypes'], nullable=cols[col]['nullable'])
        interval.append_column(tmpcol)
    

# TODO: add some fuctions to activate/desactivate constraints 0
# TODO: implement some triggers compatible (see dialects)???
    
def execute(eng, meta):
    meta.create_all(eng)

In [4]:

#con_string= 'sqlite:///test2.sqlite'
con_string= 'postgresql://postgres@localhost/Dhole'

eng, meta = og_connect(con_string, overwrite = False)

connection string: postgresql://postgres@localhost/Dhole
echo True


In [5]:
og_references(eng, meta, table_name = 'assay_certificate', key = 'SampleID', cols={'Au':{'coltypes':Float, 'nullable': True}})
og_references(eng, meta, table_name = 'rock_catalog', key = 'RockID', cols={'Description':{'coltypes':String, 'nullable': True}})
og_add_interval(eng, meta, table_name = 'assay',cols={'SampleID':{'coltypes':String, 
                                                                  'nullable': False,
                                                                  'foreignkey':{'column':'assay_certificate.SampleID',
                                                                                'ondelete':'RESTRICT',
                                                                                'onupdate':'CASCADE'}}})
og_add_interval(eng, meta, table_name = 'litho',cols={'RockID':{'coltypes':String, 
                                                                'nullable': True,
                                                                'foreignkey':{'column':'rock_catalog.RockID',
                                                                                'ondelete':'RESTRICT',
                                                                                'onupdate':'CASCADE'}}})

In [6]:
execute(eng, meta)

2017-05-19 15:30:24,128 INFO sqlalchemy.engine.base.Engine select version()
2017-05-19 15:30:24,128 INFO sqlalchemy.engine.base.Engine {}
2017-05-19 15:30:24,131 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-05-19 15:30:24,132 INFO sqlalchemy.engine.base.Engine {}
2017-05-19 15:30:24,134 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-05-19 15:30:24,135 INFO sqlalchemy.engine.base.Engine {}
2017-05-19 15:30:24,138 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-05-19 15:30:24,138 INFO sqlalchemy.engine.base.Engine {}
2017-05-19 15:30:24,141 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-05-19 15:30:24,141 INFO sqlalchemy.engine.base.Engine {}
2017-05-19 15:30:24,144 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [7]:
eng.has_table('survey_n')

2017-05-19 15:30:24,203 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2017-05-19 15:30:24,204 INFO sqlalchemy.engine.base.Engine {'name': u'survey_n'}


False

In [8]:
test_meta = MetaData()
test_meta.reflect(bind=eng)

2017-05-19 15:30:24,267 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind = 'r'
2017-05-19 15:30:24,267 INFO sqlalchemy.engine.base.Engine {'schema': u'public'}
2017-05-19 15:30:24,283 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in ('r', 'v', 'm', 'f')
        
2017-05-19 15:30:24,285 INFO sqlalchemy.engine.base.Engine {'table_name': u'survey'}
2017-05-19 15:30:24,288 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.att

In [9]:
test_meta.tables.keys()

[u'assay',
 u'rock_catalog',
 u'survey',
 u'assay_certificate',
 u'litho',
 u'collar']

In [11]:
test_meta.info

{}